In [1]:
from glob import glob
import numpy as np
import json
import os

# Label formatting

In [18]:
raw_img_paths = glob("/home/ubuntu/ForkGAN/datasets/erae/**/**/**/images/**/*.*")
raw_label_paths = glob("/home/ubuntu/ForkGAN/datasets/erae/**/**/**/annotations/**/*.*")
# raw_img_paths = glob("/home/ubuntu/ForkGAN/datasets/erae_val/**/**/**/images/**/*.*")
# raw_label_paths = glob("/home/ubuntu/ForkGAN/datasets/erae_val/**/**/**/annotations/**/*.*")
print(f"# of images: {len(raw_img_paths)}, # of labels: {len(raw_label_paths)}")
raw_img_paths.sort()
raw_label_paths.sort()
# print(raw_img_paths[:10])
# print(raw_label_paths[:10])
pass

# of images: 30561, # of labels: 29828


In [19]:
labelled_imgs_paths = []
label_paths = []
for p in raw_label_paths:
    tmp = p.replace("json","jpeg")
    tmp = tmp.replace("annotations","images")
    labelled_imgs_paths.append(tmp)
    label = p.replace("json","txt")
    label = label.replace("annotations","labels")
#     label = label.replace("erae","erae_fake")
    label_paths.append(label)
no_labelled_imgs_paths = np.setdiff1d(raw_img_paths,labelled_imgs_paths)
print(f"# images without label: {len(no_labelled_imgs_paths)}")
# print(labelled_imgs_paths)

# images without label: 733


In [10]:
# classes = ["Pedestrian", "Car", "Bus", "Truck", "Cyclist", "Dontcare", "Undefined vehicle"]
mapping = {
    "Pedestrian": 0,
    "Car": 1,
    "Bus": 2,
    "Truck": 3,
    "Cyclist": 4,
    "Dontcare": 5,
    "Undefined vehicle": 6
}

In [20]:
no_class_count = 0
corrupt_list = []
non_corrupt_label_paths = []
for i in range(len(raw_label_paths)):
    path = raw_label_paths[i]
    out_path = label_paths[i]
    tmp = json.load(open(path,))
    img_size = tmp['Image']['size'].split("x")
    label = []
    for o in tmp['Object']:
        if o['class'] not in mapping:
            print(f"o['class'] = {o['class']} not in mapping")
            no_class_count += 1
        tmp0 = (o['box2d']['x1']+o['box2d']['x2'])/(2*int(img_size[0]))
        tmp1 = (o['box2d']['y1']+o['box2d']['y2'])/(2*int(img_size[1]))
        tmp2 = (o['box2d']['x2']-o['box2d']['x1'])/int(img_size[0])
        tmp3 = (o['box2d']['y2']-o['box2d']['y1'])/int(img_size[1])
        if tmp0<0 or tmp1<0 or tmp2<0 or tmp3<0 or tmp0>1 or tmp1>1 or tmp2>1 or tmp3>1:
            corrupt_list.append(path[35:])
            label = []
            break
        s = f"{str(mapping[o['class']])} {tmp0} "
        s += f"{tmp1} "
        s += f"{tmp2} {tmp3}"
        label.append(s)
    if len(label) == 0:
        continue

#     dirname = os.path.dirname(out_path)
#     os.makedirs(dirname, exist_ok = True)
#     with open(out_path, 'w') as f:
#         for item in label:
#             f.write("%s\n" % item)
    non_corrupt_label_paths.append(path)
print(no_class_count, len(corrupt_list))

0 2787


In [17]:
# from copy import deepcopy
# non_corrupt_label_paths_fake = deepcopy(non_corrupt_label_paths)

In [21]:
with open("/home/ubuntu/ForkGAN/datasets/erae/paths.txt", 'w') as f:
# with open("/home/ubuntu/ForkGAN/datasets/erae_val/paths.txt", 'w') as f:
# with open("/home/ubuntu/ForkGAN/datasets/erae_fake/paths.txt", 'w') as f:
    for item in non_corrupt_label_paths:
#     for item in non_corrupt_label_paths+non_corrupt_label_paths_fake:
        tmp = item.replace("json","jpeg")
        tmp = tmp.replace("annotations","images")
        f.write("%s\n" % tmp)

# Night labels

In [50]:
# tmp_list = []
# tmp_list2 = []
# max_v = -1
# max_path = ''
night_label_paths = []
night_img_paths = []
for i, p in enumerate(raw_label_paths):
    tmp_i = int(p.split("/")[-4].split("_")[1])
#     tmp_list.append(tmp_i)
    if tmp_i > 11500:
        night_label_paths.append(p)
        night_img_paths.append(labelled_imgs_paths[i])
#         tmp_list2.append(tmp_i)
#     elif tmp_i > max_v:
#         max_path = p
#         max_v = tmp_i
unique_list = list(set(tmp_list))
unique_list2 = list(set(tmp_list2))
print(len(unique_list), len(unique_list2), len(night_label_paths))
# print(max_path)

1104 46 1310
/home/ubuntu/ForkGAN/datasets/erae-delivery/urban/1/2dod-urban-batch1_11314/annotations/CAM_FRONT/000000.json


# Gen paths file

In [4]:
# raw_img_paths += glob("/home/ubuntu/ForkGAN/datasets/erae_fake/**/**/**/images/**/*.*")
# with open("/home/ubuntu/ForkGAN/datasets/erae_fake/paths.txt", 'w') as f:
#     for item in raw_img_paths:
#         f.write("%s\n" % item)

# raw_img_paths = glob("/home/ubuntu/ForkGAN/datasets/erae_val/**/**/**/images/**/*.*")
# with open("/home/ubuntu/ForkGAN/datasets/erae_val/paths.txt", 'w') as f:
#     for item in raw_img_paths:
#         f.write("%s\n" % item)